**Функции будут работать, только если в датасете нет пропущенных значений**

Функция для статистических тестов на проверку соответствия распределению

In [ ]:
def statistic_test(df, alpha, test_name):
  """Функция первым аргументом принимает исследуемый датафрейм, вторым - уровень значимости,
  третьим - используемый статистический тест, сначала выделяются числовые столбцы, а затем в цикле проверяются критерием.
  Функция возвращает вывод о справедливости нулевой гипотезы для каждого числового столбца"""
  num_columns = []
  for column_name in df.columns:
        if (df[column_name].dtypes != object):
            num_columns +=[column_name]
  for column_name in num_columns:
    stat, p = test_name(df[column_name])
    if p > alpha:
      print(f'Для столбца {column_name} справедлива нулевая гипотеза, p-value = {p}')
    else:
      print(f'Для столбца {column_name} нулевая гипотеза отвергается, p-value = {p}')

In [ ]:
statistic_test.__doc__

'Функция первым аргументом принимает исследуемый датафрейм, вторым - уровень значимости,\n  третьим - используемый статистический тест, сначала выделяются числовые столбцы, а затем в цикле проверяются критерием.\n  Функция возвращает вывод о справедливости нулевой гипотезы для каждого числового столбца'

Функция для проверки стат значимости коэффициента корреляции

In [ ]:
def corr_test(df, alpha, test_name):
  """Функция первым аргументом принимает исследуемый датафрейм, вторым - уровень значимости,
  третьим - используемый статистический тест, сначала выделяются числовые столбцы, а затем в цикле проверяются критерием.
  Функция возвращает вывод о справедливости нулевой гипотезы для каждой пары столбцов"""
  num_columns = []
  for column_name in df.columns:
    if (df[column_name].dtypes != object):
            num_columns +=[column_name]
    for column_name_1 in num_columns:
      for column_name_2 in num_columns:
        stat, p = test_name(df[column_name_1], df[column_name_2])
        if p > alpha:
          print(f'столбцы {column_name_1} и {column_name_2} КОРРЕЛИРОВАНЫ, p-value = {p}, r = {stat}')
        else:
          print(f'столбцы {column_name_1} и {column_name_2} НЕ КОРРЕЛИРОВАНЫ, p-value = {p}, r = {stat}')

Функция-пайплайн для числовых и категориальных значений

Недоработана для категориальных признаков

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
def use_pipeline(df, model, metric, imput_scaler_num = None, imput_scaler_cat = None, num_scaler = None, cat_scaler = None):
  """Функция отделяет введённую целевую переменную, а предикторы предобрабатываются и поступают в модель для её обучения"""
  Y = df[input("Введите целевую переменную: ").split()]
  X = df.drop(Y.columns, axis= 1)
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=44)

  # Числовая часть пайплайна
  pipe_num = Pipeline([('imputer', imput_scaler_num()), ('scaler', num_scaler())])
  # Категориальная часть пайплайна
  pipe_cat = Pipeline([('imputer', imput_scaler_cat()), ('scaler', cat_scaler())])

  # Задаём комбинированный пайплайн
  col_transformer = ColumnTransformer([
    ('num_preproc', pipe_num, [i for i in X.columns if X[i].dtype!='object']),
    ('cat_preproc', pipe_cat, [i for i in X.columns if X[i].dtype=='object'])
    ])

  # Определяем итоговый пайплайн с учётом модели
  final_pipe = Pipeline([('preproc', col_transformer), ('model', model)])
  final_pipe.fit(X_train, Y_train)
  predict_values = final_pipe.predict(X_test)
  res = metric(Y_test, predict_values, average='macro')
  return res